
# State Space Models and Exogenous inputs

The fitness-fatigue model introduced in Modeling Cumulative Impact Part I has the form: 

$$
\text{E}(p_t) = 496 + 0.07 \sum_{i=1}^{t - 1} w_i \exp\left(\frac{-(t - i)}{\tau_1}\right) - 0.27 \sum_{i=1}^{t - 1} w_i \exp\left(\frac{-(t - i)}{\tau_2}\right),
$$
where $p_t$ is (athletic) performance and $w_t$ is training "dose" (typically time-weighted training intensity) at time $t$.

Fitness at time t is $h_t$ or $\sum_{i = 1}^{t - 1} w_i \exp(\frac{-(t - i)}{\tau_1})$ and in the previous articles of Modeling Cumulative Impact has been seen, along with fatigue $h_t$, as convolutions of training history with exponential decay. In Part II, a generalization was made to a convolution with a smooth function, but up until now the convolution has been the only tool for handling cumulative impact. In this article a second tool will be used: the system state. In order to do that, we must put the fitness-fatigue model in state-space form.

Consider the relationship of $h_t$ to $h_{t - 1}$, or $$ \sum_{i = 1}^{ t - \color{red}2} w_i \exp\left(\frac{-(t - i \color{red}- \color{red}1)}{\tau_1}\right).$$ Expanding the last term in the sum for $h_t$, we arrive at the recursion:
$$\begin{align*}
h_t &= \sum_{i = 1}^{t - 2} w_i \exp\left(\frac{-(t - i)}{\tau_1}\right) + w_{t - 1} \exp\left(\frac{-1}{\tau_1}\right) \\
    &= e^{-1 / \tau_1} h_{t - 1} + e^{-1 / \tau_1} w_{t - 1}.
\end{align*}$$
Since this is the same for fatigue $g_t$, the recursive relationship can be expressed using matrices in a "state-space form":

$$\left(\begin{matrix} 
h_t \\
g_t
\end{matrix}\right)=\left(\begin{matrix} 
e^{-1 / \tau_1}& 0 \\
0 & e^{-1 / \tau_2}
\end{matrix}\right)
\left(\begin{matrix} 
h_{t - 1} \\
g_{t - 1}
\end{matrix}\right) +
\left(\begin{matrix} 
e^{-1 / \tau_1} \\
e^{-1 / \tau_2}
\end{matrix}\right) w_{t - 1}.
$$

We can continue using matrices to express the second stage of the model with:
$$
p_t = \mu + 
\left(\begin{matrix} 
k_1 & 0 \\
0 & k_2
\end{matrix}\right)
\left(\begin{matrix} 
h_t \\
g_t
\end{matrix}\right)
+ r_t,
$$
where $r_t \sim N(0, \sigma^2).$ We have now arrived at a state-space formulation of the fitness-fatigue model, which makes available the full machinery of the Kalman Filter. But it is a Kalman Filter with exogenous inputs to the state equation, sometimes called "control inputs." 



## The Kalman Filter with control Inputs


The following is one specification of the Kalman Filter model:

$$
\begin{align*}
x_t &= A x_{t - 1} + B u_{t - 1} +  q_t, &(1)\\
y_t &= H x_t + r_t,  &(2)
\end{align*}
$$

where $q_t \sim N(0, Q)$ and $r_t \sim N(0, R)$, and the external "control" input $+ B u_{t - 1}$ may be omitted or $+ B u_{t}$ may be used instead. Such distinctions matter little when working with the model analytically.

The "Kalman" in the name comes from Rudolph Kalman, whose seminal 1960 paper [A New Approach to Linear Filtering and Prediction Problems](http://www.cs.unc.edu/~welch/kalman/kalmanPaper.html) earned him credit for the method. While the paper focuses on the general method for separating signal from noise, there [was an an almost simultaneous application](https://en.wikipedia.org/wiki/Kalman_filter#History) of the model to trajectory estimation and it was programmed into the Apollo spacecraft's computer.

Of course, rockets don't just float around passively approximately according to Newton's laws, they exert control over their state through applying thrust. Hence the control input term in the state equation. 

Compare that to the Arima model's state space representation in The Kalman Filter and (Maximum) Likelihood. There, we're treating the system dynamics as the only thing going on. That makes ARMA models pretty boring, really, but it gives them one advantage: you can predict arbitrarily long out into the future without needing to know what future inputs will be. If there are inputs driving the system, then those are just part of the system itself.

But if we're going to fit the fitness-fatigue model using the Kalman Filter, we're going to have to use control inputs. And while my last two articles, Yet Another Kalman Filter Explanation Model and The Kalman Filter and (Maximum) Likelihood, ommitted control inputs, nothing about the core logic of those articles would change if you added it in. So now that we've got the fitness-fatigue model in statespace form, we should just be able to plug it into statsmodel like before and get maximum likelihood parameters, right?

Well, not so fast. While the Kalman Filter with control inputs is the default for engineering application (see Matlab's TODO), for statistical software, you're much more likely not to see it included. For instance, in the statsmodels implementation of the Kalman Filter, there's no $B u_{t - 1}$!










As mentioned, there are recursive updating equations for the Kalman Filter that provide:

1. The likelihood of the observed data each time point given current and past observations
2. The 1-step ahead state-space prediction
3. The "fused" estimate of the state-space prediction and the measurement


are complicated (more complicated than linear regression, at least) but straightforward